In [9]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install pypdf


Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [13]:
export LANGCHAIN_TRACING_V2 = true
export LANGCHAIN_API_KEY= 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

SyntaxError: invalid syntax (2816510148.py, line 1)

In [14]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [15]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [16]:
#from langchain_community.document_loaders import WebBaseLoader
#loader = WebBaseLoader("https://fallout.fandom.com/wiki/Fallout:_New_Vegas")
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("fluigi.pdf")
pages = loader.load_and_split()

docs = pages
#docs = loader.load()

In [17]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [18]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""
Develop a verilog file that repreents the netlist for a lab on chip design.

Use only component calls like the following:   

diffmix_25px_0 mix0 (.a_fluid(soln1), .b_fluid(soln2), .out_fluid(connect0));

Use only permissible components and do not use assign statements

The following is an successfull example of an implementation



module chip_design(
  input wire soln1,
  input wire soln2,
  output wire connect0
);



  wire dilute_soln1;
  wire dilute_soln2;
  wire mix_out;

  serpentine_50px_0 dilute1 (
    .in_fluid(soln1),
    .out_fluid(dilute_soln1)
  );

  serpentine_50px_0 dilute2 (
    .in_fluid(soln2),
    .out_fluid(dilute_soln2)
  );

  diffmix_25px_0 mix0 (
    .a_fluid(dilute_soln1),
    .b_fluid(dilute_soln2),
    .out_fluid(mix_out)
  );

  serpentine_50px_0 serp0 (
    .in_fluid(mix_out),
    .out_fluid(connect0)
  );

endmodule
In this updated code, the soln1 and soln2 inputs are first passed through the serpentine_50px_0 components (dilute1 and dilute2) to reduce their concentrations by 1/2. The diluted solutions are then mixed using the diffmix_25px_0 component (mix0). Finally, the mixed solution is passed through another serpentine_50px_0 component (serp0) to further reduce its concentration by 1/2 before being output as connect0.

The following context may help: 

<context>
{context}
</context>

Question: {input}"""
)

document_chain = create_stuff_documents_chain(llm, prompt)

In [21]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [27]:
response = retrieval_chain.invoke({"input": "Could you build me three input mixer"})
print(prompt)

print(response["answer"])


input_variables=['context', 'input'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\nDevelop a verilog file that repreents the netlist for a lab on chip design.\n\nUse only component calls like the following:   \n\ndiffmix_25px_0 mix0 (.a_fluid(soln1), .b_fluid(soln2), .out_fluid(connect0));\n\nUse only permissible components and do not use assign statements\n\nThe following is an successfull example of an implementation\n\n\n\nmodule chip_design(\n  input wire soln1,\n  input wire soln2,\n  output wire connect0\n);\n\n\n\n  wire dilute_soln1;\n  wire dilute_soln2;\n  wire mix_out;\n\n  serpentine_50px_0 dilute1 (\n    .in_fluid(soln1),\n    .out_fluid(dilute_soln1)\n  );\n\n  serpentine_50px_0 dilute2 (\n    .in_fluid(soln2),\n    .out_fluid(dilute_soln2)\n  );\n\n  diffmix_25px_0 mix0 (\n    .a_fluid(dilute_soln1),\n    .b_fluid(dilute_soln2),\n    .out_fluid(mix_out)\n  );\n\n  serpentine_50px_0 serp0 (\n    .in_fluid(mix_